# ETAPA 1: Preparação e Limpeza dos Dados

## Consulta à Comunidade UFPE sobre o Uso de Inteligência Artificial

---

**Objetivo:** Garantir a qualidade e integridade dos dados antes das análises.

**Procedimentos:**
1. Remover as respostas sem consentimento de participação
2. Excluir a coluna vazia identificada ("Coluna 1")
3. Padronizar categorias de vínculo institucional
4. Verificar e tratar valores ausentes (missing data)
5. Criar variável derivada "Vinculo_Padronizado"

**Produto:** Base de dados tratada com N = 2.164 registros válidos.

**Critério de Validação:** Relatório de limpeza documentando todas as transformações realizadas.

## 1. Configuração Inicial

In [ ]:
!pip install -q pandas numpy

In [ ]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurações de exibição
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', None)

print("Bibliotecas carregadas com sucesso!")
print(f"Pandas versão: {pd.__version__}")

In [ ]:
# ============================================================================
# CONFIGURAÇÃO: Altere o caminho do arquivo conforme necessário
# ============================================================================

ARQUIVO_ENTRADA = "Consulta_a__Comunidade_UFPE_sobre_o_Uso_de_Intelige_ncia_Artificial__respostas_.xlsx"
ARQUIVO_SAIDA = "dados_limpos_etapa1.xlsx"
ARQUIVO_SAIDA_CSV = "dados_limpos_etapa1.csv"

## 2. Carregamento dos Dados Originais

In [ ]:
# Carregar dados originais
df_original = pd.read_excel(f"data/{ARQUIVO_ENTRADA}")

print("=" * 70)
print("DADOS ORIGINAIS")
print("=" * 70)
print(f"Total de registros: {len(df_original)}")
print(f"Total de colunas: {len(df_original.columns)}")
print(f"\nColunas disponíveis:")
for i, col in enumerate(df_original.columns, 1):
    print(f"  {i:2}. {col[:80]}{'...' if len(col) > 80 else ''}")

In [ ]:
# Visualizar primeiras linhas
df_original.head(3)

## 3. Procedimento 1: Remoção de Respostas sem Consentimento

In [ ]:
# Identificar coluna de consentimento
col_consentimento = 'Você leu as informações acima e concorda em participar voluntariamente desta consulta pública, ciente de que suas respostas são anônimas?'

# Verificar distribuição do consentimento
print("Distribuição do consentimento:")
print(df_original[col_consentimento].value_counts())

In [ ]:
# Filtrar apenas os que consentiram
df = df_original[df_original[col_consentimento] == 'Sim, concordo em participar'].copy()

n_removidos = len(df_original) - len(df)
print(f"Registros sem consentimento removidos: {n_removidos}")
print(f"Registros mantidos: {len(df)}")

## 4. Procedimento 2: Exclusão de Coluna Vazia

In [ ]:
# Identificar colunas completamente vazias
colunas_vazias = df.columns[df.isna().all()].tolist()
print(f"Colunas completamente vazias identificadas: {colunas_vazias}")

# Remover colunas vazias
if colunas_vazias:
    df = df.drop(columns=colunas_vazias)
    print(f"Colunas removidas: {colunas_vazias}")
    
print(f"Total de colunas após remoção: {len(df.columns)}")

## 5. Procedimento 3: Padronização de Vínculos Institucionais

In [ ]:
# Identificar coluna de vínculo
col_vinculo = 'Vínculo com a UFPE'

# Remover espaços extras
df[col_vinculo] = df[col_vinculo].str.strip()

# Verificar valores únicos antes da padronização
print(f"Valores únicos ANTES da padronização: {df[col_vinculo].nunique()}")
print("\nDistribuição original:")
print(df[col_vinculo].value_counts())

In [ ]:
# Dicionário de mapeamento para padronização
mapeamento_vinculo = {
    # Discentes de Graduação
    'Discente de Graduação': 'Discente de Graduação',
    'Discente': 'Discente de Graduação',
    'Sou egresso de bach. de Educação Física e atualmente discente de Letras': 'Discente de Graduação',
    
    # Discentes de Pós-Graduação
    'Discente de Pós-Graduação': 'Discente de Pós-Graduação',
    'Discente de Pós-Graduação & Técnico Administrativo': 'Discente de Pós-Graduação',
    'Discente de Pós-Graduação e Técnico-administrativo': 'Discente de Pós-Graduação',
    'Técnico-administrativo e discente da pós-graduação': 'Discente de Pós-Graduação',
    'Técnico-administrativo e discente de pós-graduação': 'Discente de Pós-Graduação',
    'Técnica e Discente de pós graduação': 'Discente de Pós-Graduação',
    
    # Docentes
    'Docente': 'Docente',
    'Docente aposentado.': 'Docente',
    'Docente da pós-graduação': 'Docente',
    'pós-doc': 'Docente',
    'Pós-doutor do Programa de Pós-graduação em Educação Física.': 'Docente',
    'Professor substituto': 'Docente',
    'Professor Visitante': 'Docente',
    
    # Técnico-administrativo
    'Técnico-administrativo': 'Técnico-administrativo',
    'Técnico adm e docente': 'Técnico-administrativo',
    'Técnico em Assuntos Educacionais': 'Técnico-administrativo',
    'Funcionário FADE': 'Técnico-administrativo',
    
    # Egressos
    'Egresso de Graduação': 'Egresso',
    'Egresso de Pós-Graduação': 'Egresso',
    'Egresso': 'Egresso',
    'Recém Graduado': 'Egresso',
    'Ex-discente de graduação e de pós-graduação': 'Egresso',
    'Egresso de Pós-Graduação, atualmente peaquisador em um projeto de pesquisa': 'Egresso'
}

# Criar variável padronizada
df['Vinculo_Padronizado'] = df[col_vinculo].map(mapeamento_vinculo)

print("Mapeamento aplicado com sucesso!")

In [ ]:
# Verificar se há valores não mapeados
nao_mapeados = df[df['Vinculo_Padronizado'].isna()][col_vinculo].unique()

if len(nao_mapeados) > 0:
    print("⚠️ ATENÇÃO: Valores não mapeados encontrados:")
    for v in nao_mapeados:
        print(f"   - '{v}'")
    print("\nAdicione estes valores ao dicionário de mapeamento acima.")
else:
    print("✓ Todos os valores foram mapeados com sucesso!")

In [ ]:
# Verificar distribuição após padronização
print(f"Valores únicos APÓS padronização: {df['Vinculo_Padronizado'].nunique()}")
print("\nDistribuição padronizada:")

dist_vinculo = df['Vinculo_Padronizado'].value_counts()
for vinculo, count in dist_vinculo.items():
    pct = count / len(df) * 100
    print(f"   • {vinculo}: {count} ({pct:.1f}%)")

print(f"\n   TOTAL: {len(df)} registros")

## 6. Procedimento 4: Renomeação de Variáveis

In [ ]:
# Dicionário de renomeação para nomes mais curtos e padronizados
nomes_curtos = {
    'Carimbo de data/hora': 'Timestamp',
    col_consentimento: 'Consentimento',
    'Vínculo com a UFPE': 'Vinculo_Original',
    'Centro, Unidade Acadêmica ou Unidade Gestora': 'Centro_Unidade',
    'Curso (para discentes)': 'Curso',
    'Você já utilizou alguma ferramenta de IA (como ChatGPT, Copilot, Gemini, Claude, Perplexity etc.)?': 'Frequencia_Uso_IA',
    'Em quais contextos você utiliza ou pretende utilizar ferramentas de IA?': 'Contextos_Uso',
    'Quais benefícios você percebe no uso da IA na UFPE?': 'Beneficios_Percebidos',
    'Quais riscos ou preocupações você identifica no uso da IA na universidade?': 'Riscos_Preocupacoes',
    'O uso que você faz (ou faria) de ferramentas de IA é predominantemente:': 'Tipo_Uso',
    'Avalie as afirmações abaixo de acordo com seu grau de concordância. [A inteligência artificial já não é mais uma opção, é uma realidade com a qual precisamos aprender a lidar.]': 'Likert_IA_Realidade',
    'Avalie as afirmações abaixo de acordo com seu grau de concordância. [O uso ético e responsável da inteligência artificial requer diretrizes e normativas claras.]': 'Likert_Diretrizes_Eticas',
    'Avalie as afirmações abaixo de acordo com seu grau de concordância. [A UFPE deve oferecer diretrizes sobre o uso de IA em trabalhos e avaliações de atividades dos discentes.]': 'Likert_Diretrizes_Avaliacoes',
    'Avalie as afirmações abaixo de acordo com seu grau de concordância. [O uso de IA deve ser incentivado como ferramenta de apoio, desde que usado de forma responsável.]': 'Likert_Incentivo_IA',
    'Avalie as afirmações abaixo de acordo com seu grau de concordância. [A universidade deve promover mais debates, orientações de uso e espaços de reflexão sobre os impactos da IA na sociedade.]': 'Likert_Debates_Impactos',
    'Na sua percepção, como a IA pode impactar a produção autoral?': 'Impacto_Producao_Autoral',
    'Na sua percepção, como a IA pode impactar a integridade acadêmica?': 'Impacto_Integridade_Academica',
    'Que medidas a UFPE deveria adotar para garantir o uso ético, transparente e responsável da IA?': 'Medidas_Uso_Etico',
    'Quais temas devem ser priorizados no Plano Institucional de IA da UFPE?': 'Temas_Prioritarios',
    'Como a UFPE deve estruturar sua governança em relação à IA?': 'Estrutura_Governanca',
    'Você se sente preparado(a) para usar ferramentas de IA de forma ética e eficaz?': 'Preparacao_Uso_Etico',
    'Que tipos de formação ou de suporte institucional você considera mais importantes?': 'Tipos_Formacao',
    'Deixe aqui sugestões ou comentários finais.': 'Comentarios_Finais'
}

# Aplicar renomeação
df = df.rename(columns=nomes_curtos)

print("Colunas renomeadas:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2}. {col}")

## 7. Procedimento 5: Análise de Valores Ausentes (Missing Data)

In [ ]:
# Análise de missing values
print("=" * 70)
print("ANÁLISE DE VALORES AUSENTES")
print("=" * 70)

missing_report = []
for col in df.columns:
    n_missing = df[col].isna().sum()
    pct_missing = (n_missing / len(df)) * 100
    missing_report.append({
        'Variável': col,
        'N_Missing': n_missing,
        'Pct_Missing': round(pct_missing, 1)
    })

# Criar DataFrame do relatório
df_missing = pd.DataFrame(missing_report)
df_missing = df_missing.sort_values('N_Missing', ascending=False)

# Exibir apenas variáveis com missing
print("\nVariáveis com valores ausentes:")
print("-" * 50)
df_com_missing = df_missing[df_missing['N_Missing'] > 0]
print(df_com_missing.to_string(index=False))

print(f"\nVariáveis sem valores ausentes: {len(df_missing[df_missing['N_Missing'] == 0])}")

In [ ]:
# Visualização dos missing values
import matplotlib.pyplot as plt

# Filtrar apenas variáveis com missing > 0
df_plot = df_com_missing[df_com_missing['Pct_Missing'] > 0].copy()

if len(df_plot) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.barh(df_plot['Variável'], df_plot['Pct_Missing'], color='steelblue')
    ax.set_xlabel('% de Valores Ausentes')
    ax.set_title('Valores Ausentes por Variável')
    ax.invert_yaxis()
    
    # Adicionar labels nas barras
    for bar, pct in zip(bars, df_plot['Pct_Missing']):
        ax.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, 
                f'{pct}%', va='center', fontsize=9)
    
    plt.tight_layout()
    plt.savefig('missing_values_etapa1.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\nGráfico salvo como: missing_values_etapa1.png")
else:
    print("Nenhuma variável com valores ausentes para plotar.")

## 8. Resumo da Etapa 1

In [ ]:
print("=" * 70)
print("RESUMO DA ETAPA 1 - LIMPEZA CONCLUÍDA")
print("=" * 70)

print(f"""
📋 TRANSFORMAÇÕES REALIZADAS:

   1. Remoção de registros sem consentimento
      • Registros removidos: {n_removidos}
      • Registros mantidos: {len(df)}

   2. Exclusão de coluna vazia
      • Coluna removida: 'Coluna 1'
      • Total de colunas: {len(df.columns)}

   3. Padronização de vínculos institucionais
      • Categorias originais: 26
      • Categorias padronizadas: {df['Vinculo_Padronizado'].nunique()}
""")

for vinculo, count in df['Vinculo_Padronizado'].value_counts().items():
    pct = count / len(df) * 100
    print(f"        - {vinculo}: {count} ({pct:.1f}%)")

print(f"""
   4. Renomeação de colunas para facilitar análise
      • {len(df.columns)} colunas renomeadas com nomes curtos

   5. Documentação de valores ausentes
      • Variáveis com missing identificadas e documentadas

📁 BASE DE DADOS FINAL:
   • Registros válidos: {len(df)}
   • Variáveis: {len(df.columns)}
""")

## 9. Salvar Base de Dados Limpa

In [ ]:
# Salvar em Excel
df.to_excel(ARQUIVO_SAIDA, index=False)
print(f"✅ Base de dados limpa salva em: {ARQUIVO_SAIDA}")

# Salvar em CSV (opcional)
df.to_csv(ARQUIVO_SAIDA_CSV, index=False)
print(f"✅ Base de dados limpa salva em: {ARQUIVO_SAIDA_CSV}")

In [ ]:
# Visualizar amostra da base final
print("Amostra da base de dados limpa:")
df.head()

## 10. Geração do Relatório de Limpeza

In [ ]:
# Gerar relatório de limpeza em Markdown
relatorio = f"""# RELATÓRIO DE LIMPEZA DE DADOS
## Etapa 1 — Consulta à Comunidade UFPE sobre o Uso de IA

**Data de execução:** {datetime.now().strftime("%d/%m/%Y às %H:%M")}

---

## 1. DADOS ORIGINAIS

| Métrica | Valor |
|---------|-------|
| Total de registros | {len(df_original)} |
| Total de colunas | {len(df_original.columns)} |

---

## 2. TRANSFORMAÇÕES REALIZADAS

### 2.1 Remoção de registros sem consentimento

| Descrição | Quantidade |
|-----------|------------|
| Registros sem consentimento removidos | {n_removidos} |
| Registros mantidos | {len(df)} |
| Taxa de exclusão | {n_removidos/len(df_original)*100:.1f}% |

### 2.2 Exclusão de coluna vazia

| Coluna removida | Motivo |
|-----------------|--------|
| "Coluna 1" | Completamente vazia (100% missing) |

### 2.3 Padronização de vínculos institucionais

**De 26 categorias originais para {df['Vinculo_Padronizado'].nunique()} categorias padronizadas:**

| Categoria Padronizada | Registros | % |
|-----------------------|-----------|---|
"""

for vinculo, count in df['Vinculo_Padronizado'].value_counts().items():
    pct = count / len(df) * 100
    relatorio += f"| {vinculo} | {count} | {pct:.1f}% |\n"

relatorio += f"""| **TOTAL** | **{len(df)}** | **100%** |

---

## 3. ANÁLISE DE VALORES AUSENTES

| Variável | N Missing | % Missing |
|----------|-----------|----------|
"""

for _, row in df_com_missing.iterrows():
    relatorio += f"| {row['Variável']} | {row['N_Missing']} | {row['Pct_Missing']}% |\n"

relatorio += f"""
**Variáveis completas (0% missing):** {len(df_missing[df_missing['N_Missing'] == 0])} variáveis

---

## 4. BASE DE DADOS FINAL

| Métrica | Valor |
|---------|-------|
| Total de registros válidos | {len(df)} |
| Total de variáveis | {len(df.columns)} |
| Arquivo gerado | {ARQUIVO_SAIDA} |

---

## 5. VALIDAÇÃO

- [x] Todos os {len(df)} registros possuem consentimento válido
- [x] Coluna vazia removida
- [x] 100% dos vínculos mapeados para categorias padronizadas
- [x] Nomes de variáveis padronizados
- [x] Missing values documentados

---

*Relatório gerado automaticamente em {datetime.now().strftime("%d/%m/%Y às %H:%M")}*
"""

# Salvar relatório
with open("relatorio_limpeza_etapa1.md", "w", encoding='utf-8') as f:
    f.write(relatorio)

print("✅ Relatório de limpeza salvo em: relatorio_limpeza_etapa1.md")
print("\n" + "=" * 70)
print("ETAPA 1 CONCLUÍDA COM SUCESSO!")
print("=" * 70)

---

## Arquivos Gerados

| Arquivo | Descrição |
|---------|----------|
| `dados_limpos_etapa1.xlsx` | Base de dados tratada |
| `dados_limpos_etapa1.csv` | Base de dados tratada (CSV) |
| `relatorio_limpeza_etapa1.md` | Relatório documentando transformações |
| `missing_values_etapa1.png` | Gráfico de valores ausentes |

---

**Próxima etapa:** Etapa 2 - Análise do Perfil dos Respondentes